In [313]:
from scipy.io import arff
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
import pickle

In [314]:
# Load the ARFF file
data, meta = arff.loadarff('Seattle_Crime_Data_06-23-2019-4.arff')

# Convert to a pandas DataFrame
df = pd.DataFrame(data)

# Decode byte strings (ARFF stores nominal/string attributes as bytes)
for col in df.select_dtypes([object]):
    df[col] = df[col].str.decode('utf-8')
    
new_df = df.copy()

In [315]:
print(df.head())
print(df.shape)

   Report_Number  Occurred_Time  Reported_Time     Crime_Subcategory  \
0   1.975000e+12          900.0         1500.0  BURGLARY-RESIDENTIAL   
1   1.976000e+12            1.0         2359.0     SEX OFFENSE-OTHER   
2   1.979000e+12         1600.0         1430.0             CAR PROWL   
3   1.981000e+13         2029.0         2030.0              HOMICIDE   
4   1.981000e+12         2000.0          435.0  BURGLARY-RESIDENTIAL   

    Primary_Offense_Description   Precinct Sector Beat  \
0            BURGLARY-FORCE-RES      SOUTH      R   R3   
1     SEXOFF-INDECENT LIBERTIES    UNKNOWN      ?    ?   
2                THEFT-CARPROWL       EAST      G   G2   
3  HOMICIDE-PREMEDITATED-WEAPON      SOUTH      S   S2   
4            BURGLARY-FORCE-RES  SOUTHWEST      W   W3   

                     Neighborhood  
0            LAKEWOOD/SEWARD PARK  
1                         UNKNOWN  
2        CENTRAL AREA/SQUIRE PARK  
3                 BRIGHTON/DUNLAP  
4  ROXHILL/WESTWOOD/ARBOR HEIGHTS  
(5

# Prepreprocessing

In [316]:
# time validation
print((new_df['Occurred_Time'] > 2359).sum())  # should be 0
print((new_df['Occurred_Time'] < 0).sum())     # should be 0

# check for reversed times
reversed_times = new_df['Reported_Time'] < new_df['Occurred_Time']
print(f"Reversed times: {reversed_times.sum()}")

# check class distribution
print(new_df['Primary_Offense_Description'].value_counts())

0
0
Reversed times: 140045
Primary_Offense_Description
THEFT-CARPROWL                   131297
THEFT-SHOPLIFT                    48638
THEFT-OTH                         47275
VEH-THEFT-AUTO                    37840
BURGLARY-FORCE-RES                27984
                                  ...  
NARC-SMUGGLE-HEROIN                   1
HOMICIDE-NEG-MANS-GUN                 1
NARC-SELL-BARBITUATE                  1
NARC-MANUFACTURE-HALLUCINOGEN         1
HOMICIDE-NEG-MANS-WEAPON              1
Name: count, Length: 144, dtype: int64


# Preprocessing

## 1. handle missing values

In [317]:
new_df = new_df.replace(['UNKNOWN', '?', 'Unknown', 'unknown'], pd.NA)
print(new_df.isnull().sum()[new_df.isnull().sum() > 0]) 

Occurred_Time           2
Reported_Time           2
Crime_Subcategory     262
Precinct             3352
Sector               3346
Beat                 3298
Neighborhood         3366
dtype: int64


Because `Occured_Time` and `Reported_Time` are only missing for two rows (a small number as we have 523590 rows), we can drop them.

In [318]:
# drop rows with missing Occurred_Time or Reported_Time
new_df = new_df.dropna(subset=['Occurred_Time', 'Reported_Time'])

For missing values in other columns, we can substitute them as a unified "UNKNOWN" category.

In [319]:
# For missing values in other columns, we can substitute them as a unified "UNKNOWN" category.
new_df = new_df.fillna('UNKNOWN')
print(new_df.isnull().sum()[new_df.isnull().sum() > 0]) 

Series([], dtype: int64)


## 2. get rid of Report_Number and it's a primary key (to link multiple offenses to one report)

In [320]:
new_df = new_df.drop(columns=['Report_Number'])

## 3. drop Crime_Subcategory to prevent leakage

In [321]:
new_df = new_df.drop(columns=["Crime_Subcategory"])

## 4. split occurred_time and reported_time into hour and minute and maybe sin/cos tranform them to keep the relationship that 00:00 comes after 23:59

In [322]:
# split "Reported_Time" into "reported_hour" and "reported_minute"
new_df["reported_hour"] = new_df["Reported_Time"].apply(lambda x: int(x/100) if not pd.isna(x) else pd.NA)
new_df["reported_minute"] = new_df["Reported_Time"].apply(lambda x: x % 100 if not pd.isna(x) else pd.NA)
# make hours and minutes integers
new_df["reported_hour"] = new_df["reported_hour"].astype("Int64")
new_df["reported_minute"] = new_df["reported_minute"].astype("Int64")
# cyclical encoding (sin/cos)
new_df["reported_hour_sin"] = np.sin(2 * np.pi * new_df["reported_hour"] / 24)
new_df["reported_hour_cos"] = np.cos(2 * np.pi * new_df["reported_hour"] / 24)
new_df["reported_minute_sin"] = np.sin(2 * np.pi * new_df["reported_minute"] / 60)
new_df["reported_minute_cos"] = np.cos(2 * np.pi * new_df["reported_minute"] / 60)

# split "Occurred_Time" into "occured_hour" and "occured_minute"
new_df["occured_hour"] = new_df["Occurred_Time"].apply(lambda x: int(x/100) if not pd.isna(x) else pd.NA)
new_df["occured_minute"] = new_df["Occurred_Time"].apply(lambda x: x % 100 if not pd.isna(x) else pd.NA)
# make hours and minutes integers
new_df["occured_hour"] = new_df["occured_hour"].astype("Int64")
new_df["occured_minute"] = new_df["occured_minute"].astype("Int64")
# cyclical encoding (sin/cos)
new_df["occured_hour_sin"] = np.sin(2 * np.pi * new_df["occured_hour"] / 24)
new_df["occured_hour_cos"] = np.cos(2 * np.pi * new_df["occured_hour"] / 24)
new_df["occured_minute_sin"] = np.sin(2 * np.pi * new_df["occured_minute"] / 60)
new_df["occured_minute_cos"] = np.cos(2 * np.pi * new_df["occured_minute"] / 60)

# drop Reported_Time, Occurred_Time, reported_hour, reported_minute, occured_hour, occured_minute
new_df = new_df.drop(columns=["Reported_Time", "Occurred_Time", "reported_hour", "reported_minute", "occured_hour", "occured_minute"])

new_df

,Primary_Offense_Description,Precinct,Sector,Beat,Neighborhood,reported_hour_sin,reported_hour_cos,reported_minute_sin,reported_minute_cos,occured_hour_sin,occured_hour_cos,occured_minute_sin,occured_minute_cos
0,BURGLARY-FORCE-RES,SOUTH,R,R3,LAKEWOOD/SEWARD PARK,-0.707107,-0.707107,0.0,1.0,0.707107,-0.707107,0.0,1.0
1,SEXOFF-INDECENT LIBERTIES,UNKNOWN,UNKNOWN,UNKNOWN,UNKNOWN,-0.258819,0.965926,-0.104528,0.994522,0.0,1.0,0.104528,0.994522
2,THEFT-CARPROWL,EAST,G,G2,CENTRAL AREA/SQUIRE PARK,-0.5,-0.866025,0.0,-1.0,-0.866025,-0.5,0.0,1.0
3,HOMICIDE-PREMEDITATED-WEAPON,SOUTH,S,S2,BRIGHTON/DUNLAP,-0.866025,0.5,0.0,-1.0,-0.866025,0.5,0.104528,-0.994522
4,BURGLARY-FORCE-RES,SOUTHWEST,W,W3,ROXHILL/WESTWOOD/ARBOR HEIGHTS,0.866025,0.5,-0.5,-0.866025,-0.866025,0.5,0.0,1.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...
523585,CHILD-OTHER,SOUTH,O,O3,MID BEACON HILL,-0.965926,-0.258819,0.978148,0.207912,-0.965926,-0.258819,0.978148,0.207912
523586,BURGLARY-FORCE-RES,EAST,C,C2,MONTLAKE/PORTAGE BAY,-0.965926,-0.258819,0.809017,-0.587785,0.965926,-0.258819,0.0,-1.0
523587,ROBBERY-BUSINESS-BODYFORCE,SOUTH,S,S2,RAINIER BEACH,-0.965926,-0.258819,0.587785,-0.809017,-0.965926,-0.258819,0.587785,-0.809017
523588,THEFT-SHOPLIFT,NORTH,L,L2,NORTHGATE,-0.965926,0.258819,0.406737,0.913545,-0.965926,-0.258819,-0.866025,0.5


## 5. handle class imbalance

We have two problems:
- there are classes with tens of thousands of observations, but also classes with only one observations - huge imbalance
- there are 144 classes, which is a lot of classes

We're going to solve this by merging the rare classes (with less than N observations) into one "OTHER" class. The remaining imbalance will be solved by the classifier used, becase for example for Support Vector Machines it has no effect oversampling the minority classes and SVMs handle imbalances quite well on their own.

In [323]:
# check class distribution
class_counts = new_df['Primary_Offense_Description'].value_counts()
print(class_counts)
RARITY_THRESHOLD = 1000
rare_classes = class_counts[class_counts < RARITY_THRESHOLD].index
print(f"Number of rare classes (less than {RARITY_THRESHOLD} instances): {len(rare_classes)}")
new_df['Primary_Offense_Description'] = new_df['Primary_Offense_Description'].apply(lambda x: 'OTHER' if x in rare_classes else x)

print(new_df['Primary_Offense_Description'].value_counts())

Primary_Offense_Description
THEFT-CARPROWL                   131297
THEFT-SHOPLIFT                    48637
THEFT-OTH                         47275
VEH-THEFT-AUTO                    37840
BURGLARY-FORCE-RES                27984
                                  ...  
NARC-SMUGGLE-HEROIN                   1
HOMICIDE-NEG-MANS-GUN                 1
NARC-SELL-BARBITUATE                  1
NARC-MANUFACTURE-HALLUCINOGEN         1
HOMICIDE-NEG-MANS-WEAPON              1
Name: count, Length: 144, dtype: int64
Number of rare classes (less than 1000 instances): 97
Primary_Offense_Description
THEFT-CARPROWL                    131297
THEFT-SHOPLIFT                     48637
THEFT-OTH                          47275
VEH-THEFT-AUTO                     37840
BURGLARY-FORCE-RES                 27984
THEFT-BUILDING                     21438
BURGLARY-NOFORCE-RES               18859
TRESPASS                           17722
OTHER                              17296
BURGLARY-FORCE-NONRES              15160
D

## 6. split the data into train and test

In [324]:
X = new_df.drop(columns=['Primary_Offense_Description'])
y = new_df['Primary_Offense_Description']

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42, stratify=y)

print(f"Train shape: {X_train.shape}, Test shape: {X_test.shape}")

Train shape: (418870, 12), Test shape: (104718, 12)


## 6. one-hot encode nominal features

In [325]:
# encode separately
X_train = pd.get_dummies(X_train, columns=['Precinct', 'Sector', 'Beat', 'Neighborhood'])
X_test = pd.get_dummies(X_test, columns=['Precinct', 'Sector', 'Beat', 'Neighborhood'])

# align (ensures same columns)
X_train, X_test = X_train.align(X_test, join='left', axis=1, fill_value=0)

NOTE: *For this dataset, there are no continuous features to scale, therefore we're not going to do any scaling.*

In [326]:
X_train

,reported_hour_sin,reported_hour_cos,reported_minute_sin,reported_minute_cos,occured_hour_sin,occured_hour_cos,occured_minute_sin,occured_minute_cos,Precinct_EAST,Precinct_NORTH,...,Neighborhood_ROXHILL/WESTWOOD/ARBOR HEIGHTS,Neighborhood_SANDPOINT,Neighborhood_SLU/CASCADE,Neighborhood_SODO,Neighborhood_SOUTH BEACON HILL,Neighborhood_SOUTH DELRIDGE,Neighborhood_SOUTH PARK,Neighborhood_UNIVERSITY,Neighborhood_UNKNOWN,Neighborhood_WALLINGFORD
292102,0.965926,-0.258819,0.587785,-0.809017,0.965926,-0.258819,0.866025,-0.5,False,False,...,False,False,False,False,False,False,False,False,False,False
394717,-0.5,-0.866025,0.951057,0.309017,-0.5,-0.866025,0.951057,0.309017,False,True,...,False,False,False,False,False,False,False,False,False,False
143453,0.0,1.0,0.587785,-0.809017,-0.258819,0.965926,0.0,-1.0,False,False,...,False,False,False,False,False,False,False,False,False,False
357712,-0.965926,0.258819,-0.5,-0.866025,0.5,0.866025,1.0,0.0,False,False,...,False,False,False,False,False,False,False,False,False,False
50347,-1.0,-0.0,0.0,1.0,-0.965926,-0.258819,0.0,1.0,False,False,...,False,False,False,False,False,False,False,False,False,False
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
417055,-0.258819,-0.965926,0.104528,0.994522,0.5,0.866025,0.0,-1.0,False,False,...,False,False,False,False,False,False,True,False,False,False
138596,-0.707107,-0.707107,-0.866025,-0.5,-0.866025,0.5,0.0,-1.0,False,False,...,False,False,False,False,False,False,False,False,False,False
135165,0.0,-1.0,-0.978148,0.207912,0.0,-1.0,0.0,1.0,False,True,...,False,True,False,False,False,False,False,False,False,False
29239,0.258819,0.965926,-0.809017,-0.587785,-0.258819,0.965926,-0.994522,0.104528,False,False,...,False,False,False,False,False,False,False,False,False,False


In [327]:
X_test

,reported_hour_sin,reported_hour_cos,reported_minute_sin,reported_minute_cos,occured_hour_sin,occured_hour_cos,occured_minute_sin,occured_minute_cos,Precinct_EAST,Precinct_NORTH,...,Neighborhood_ROXHILL/WESTWOOD/ARBOR HEIGHTS,Neighborhood_SANDPOINT,Neighborhood_SLU/CASCADE,Neighborhood_SODO,Neighborhood_SOUTH BEACON HILL,Neighborhood_SOUTH DELRIDGE,Neighborhood_SOUTH PARK,Neighborhood_UNIVERSITY,Neighborhood_UNKNOWN,Neighborhood_WALLINGFORD
441136,-0.258819,-0.965926,0.5,-0.866025,0.0,1.0,0.0,1.0,False,False,...,False,False,False,False,False,False,False,False,False,False
97340,-0.258819,0.965926,0.309017,0.951057,-0.965926,0.258819,0.0,-1.0,False,False,...,False,False,False,False,False,False,False,False,False,False
174480,-0.965926,0.258819,-0.669131,0.743145,0.5,-0.866025,-1.0,-0.0,False,False,...,False,False,False,False,False,False,False,False,False,False
429822,0.965926,-0.258819,-0.406737,0.913545,-0.965926,-0.258819,0.0,-1.0,False,True,...,False,False,False,False,False,False,False,False,False,False
286495,0.5,-0.866025,0.994522,-0.104528,-1.0,-0.0,0.0,1.0,True,False,...,False,False,False,False,False,False,False,False,False,False
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
232291,0.707107,-0.707107,-0.104528,-0.994522,-0.965926,0.258819,0.0,1.0,False,True,...,False,False,False,False,False,False,False,False,False,False
237500,-0.965926,-0.258819,-0.104528,0.994522,0.0,-1.0,-0.866025,-0.5,False,False,...,False,False,False,False,False,False,False,False,False,False
191577,-0.866025,-0.5,0.669131,0.743145,-0.866025,-0.5,0.5,0.866025,True,False,...,False,False,False,False,False,False,False,False,False,False
483929,-0.707107,-0.707107,-0.809017,-0.587785,-0.965926,0.258819,0.866025,0.5,False,False,...,False,False,False,False,False,False,False,False,False,False


In [328]:
print(f"Final number of classes: {y.nunique()}")
print(y.value_counts())

Final number of classes: 48
Primary_Offense_Description
THEFT-CARPROWL                    131297
THEFT-SHOPLIFT                     48637
THEFT-OTH                          47275
VEH-THEFT-AUTO                     37840
BURGLARY-FORCE-RES                 27984
THEFT-BUILDING                     21438
BURGLARY-NOFORCE-RES               18859
TRESPASS                           17722
OTHER                              17296
BURGLARY-FORCE-NONRES              15160
DUI-LIQUOR                         11490
THEFT-BICYCLE                      11091
ASSLT-AGG-WEAPON                    9737
BURGLARY-SECURE PARKING-RES         8852
BURGLARY-NOFORCE-NONRES             8371
THEFT-AUTOACC                       8367
ROBBERY-STREET-BODYFORCE            8108
THEFT-LICENSE PLATE                 7588
CHILD-OTHER                         4608
ASSLT-AGG-DV-WEAPON                 3954
ASSLT-AGG-BODYFORCE                 3655
THEFT-MAIL                          3579
NARC-POSSESS-COCAINE                3351
V

# Save datasets as Parquet

In [330]:
X_train = X_train.astype({col: 'float64' for col in X_train.select_dtypes(include=['Float64']).columns})
X_test = X_test.astype({col: 'float64' for col in X_test.select_dtypes(include=['Float64']).columns})

X_train.to_parquet('X_train.parquet', compression='snappy', index=False)
X_test.to_parquet('X_test.parquet', compression='snappy', index=False)
y_train.to_frame('Primary_Offense_Description').to_parquet('y_train.parquet', compression='snappy', index=False)
y_test.to_frame('Primary_Offense_Description').to_parquet('y_test.parquet', compression='snappy', index=False)


ArrowKeyError: A type extension with name pandas.period already defined